# Calibrate AVHRR NDVI to match MODIS NDVI


### Load packages
Import Python packages that are used for the analysis.

In [ ]:
%matplotlib inline

import xarray as xr
import numpy as np
import pandas as pd
import seaborn as sb
from joblib import dump
from scipy import stats
import geopandas as gpd
from pprint import pprint
from odc.geo.xr import assign_crs
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

import lightgbm as lgbm
from lightgbm import LGBMRegressor

import shap
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import sys
sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools/')
from dea_tools.spatial import xr_rasterize

sys.path.append('/g/data/os22/chad_tmp/AusEFlux/src/')
from _collect_prediction_data import round_coords

import warnings
warnings.filterwarnings("ignore")

### Analysis parameters


In [ ]:
model_var='LST'
n_samples = 30000
n_val = 5000
name = 'trees'
t1 = '1982'
t2 = '2013'

## Assemble datasets for training and predicting

### Clean/filter AVHRR data

In [ ]:
ds = xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/data/'+model_var+'_harmonization/AVHRR_'+model_var+'_5km_monthly_1982_2013.nc')
ds = assign_crs(ds, crs ='epsg:4326')

if model_var=='LST':
    ds = ds.rename({'VZA_median':'SZEN_median'})

In [ ]:
before_fraction_avail = (~np.isnan(ds[model_var+'_avhrr'])).sum('time')/len(ds.time)

In [ ]:
#filter by num of obs/month
# ds = ds.where(ds['n_obs']>=2)

#remove any very low NDVI vals
if model_var=='NDVI':
    ds = ds.where(ds[model_var+'_avhrr']>=0.01)

# filter by coefficient of variation each month
ds[model_var+'_cv'] = ds[model_var+'_stddev'] / ds[model_var+'_avhrr']
ds = ds.where(ds[model_var+'_cv']<0.5)

#filter by large std dev anomalies
def stand_anomalies(ds, clim_mean, clim_std):
    std_anom = xr.apply_ufunc(lambda x, m, s: (x - m) / s,
    ds.compute().groupby("time.month"),
    clim_mean, clim_std)
    return std_anom

#calculate anomalies
clim_std = ds.groupby('time.month').std()
clim = ds.groupby('time.month').mean()
std_anom = stand_anomalies(ds, clim, clim_std)

#create masks where values are < 3.5 stddev >
anom_mask = xr.where((std_anom[model_var+'_avhrr'] > -4) & (std_anom[model_var+'_avhrr'] < 4), 1, 0)
sza_anom_mask = xr.where((std_anom['SZEN_median'] > -4) & (std_anom['SZEN_median'] < 4), 1, 0)
tod_anom_mask = xr.where((std_anom['TIMEOFDAY_median'] > -4) & (std_anom['TIMEOFDAY_median'] < 4), 1, 0)
ds = ds.where(anom_mask)
ds = ds.where(sza_anom_mask)
ds = ds.where(tod_anom_mask)


### Plot available fraction of data before/after filtering

In [ ]:
after_fraction_avail = (~np.isnan(ds[model_var+'_avhrr'])).sum('time')/len(ds.time)

fract_avail_mask = xr.where(before_fraction_avail>0, 1, 0)
after_fraction_avail = after_fraction_avail.where(fract_avail_mask)
before_fraction_avail = before_fraction_avail.where(fract_avail_mask)

fig,ax=plt.subplots(1,2, figsize=(10,4),sharey=True)
before_fraction_avail.rename('').plot.imshow(robust=True, ax=ax[0], cmap='magma', add_labels=False)
after_fraction_avail.rename('').plot.imshow(robust=True, ax=ax[1], cmap='magma', add_labels=False)
ax[0].set_title('Mean Fraction (before): '+str(round(before_fraction_avail.mean().values.item(), 3)));
ax[1].set_title('Mean Fraction (after): '+str(round(after_fraction_avail.mean().values.item(), 3)))
plt.tight_layout();

fig.savefig("/g/data/os22/chad_tmp/climate-carbon-interactions/results/figs/AVHRR_"+model_var+"_fraction_available.png",
            bbox_inches='tight', dpi=300)

### export filtered mask

In [ ]:
filtered_mask = ~np.isnan(ds[model_var+'_avhrr'])
filtered_mask = filtered_mask.drop('month')
filtered_mask.to_netcdf('/g/data/os22/chad_tmp/climate-carbon-interactions/data/'+model_var+'_harmonization/AVHRR_'+model_var+'_filtered_mask_1982_2013.nc')

### Add lagged features

In [ ]:
ds[model_var+'_avhrr_1f'] = ds[model_var+'_avhrr'].shift(time=1)
ds[model_var+'_avhrr_1b'] = ds[model_var+'_avhrr'].shift(time=-1)
# ds['NDVI_avhrr_2f'] = ds['NDVI_avhrr'].shift(time=2)
# ds['NDVI_avhrr_2b'] = ds['NDVI_avhrr'].shift(time=-2)

In [ ]:
ds = ds.sel(time=slice(t1, t2))
ds = ds.drop('month')

### Open covariables

In [ ]:
base = '/g/data/os22/chad_tmp/climate-carbon-interactions/data/'

if model_var=='NDVI':
    datasets = [
        model_var+'harmonization/MODIS_'+model_var+'_5km_monthly_200003_202212.nc',
        '5km/MOY_5km_monthly_1982_2022.nc',
        '5km/srad_5km_monthly_1982_2022.nc',
        '5km/srad_anom_5km_monthly_1982_2022.nc',
        '5km/rain_cml3_5km_monthly_1982_2022.nc',
        '5km/rain_cml3_anom_5km_monthly_1982_2022.nc',
        # '5km/rain_5km_monthly_1981_2022.nc',
        # '5km/rain_cml6_anom_5km_monthly_1982_2022.nc',
        # '5km/vpd_5km_monthly_1982_2022.nc',
        # '5km/tavg_5km_monthly_1982_2022.nc',
        # '5km/tavg_anom_5km_monthly_1982_2022.nc',
               ]

if model_var=='LST':
    datasets = [
        model_var+'_harmonization/MODIS_'+model_var+'_5km_monthly_200003_202212.nc',
        '5km/MOY_5km_monthly_1982_2022.nc',
        '5km/srad_5km_monthly_1982_2022.nc',
        '5km/srad_anom_5km_monthly_1982_2022.nc',
        # '5km/rain_cml3_5km_monthly_1982_2022.nc',
        # '5km/rain_cml3_anom_5km_monthly_1982_2022.nc',
        # '5km/rain_5km_monthly_1981_2022.nc',
        # '5km/rain_cml6_anom_5km_monthly_1982_2022.nc',
        # '5km/vpd_5km_monthly_1982_2022.nc',
        '5km/tavg_5km_monthly_1982_2022.nc',
        '5km/tavg_anom_5km_monthly_1982_2022.nc',
               ]

In [ ]:
dss = []
names = []
for d in datasets:
    xx =  assign_crs(xr.open_dataset(base+d),crs='epsg:4326').sel(time=slice(t1,t2))
    xx = xx.drop('spatial_ref')
    names.append(list(xx.data_vars)[0])
    dss.append(xx.transpose('time', 'latitude', 'longitude'))

covars = xr.merge(dss)
covars = assign_crs(covars, crs ='epsg:4326')
covars = covars.rename({model_var+'_median':model_var+'_modis'})

#merge the AVHR with covariables
ds = xr.merge([ds,covars])

### Add some MODIS summary stats

In [ ]:
mean_modis = ds[model_var+'_modis'].mean('time')
mean_modis = mean_modis.expand_dims(time=ds.time)
ds[model_var+'_modis_mean'] = mean_modis

min_modis = ds[model_var+'_modis'].quantile(0.05, dim='time').drop('quantile')
min_modis = min_modis.expand_dims(time=ds.time)
ds[model_var+'_modis_min'] = min_modis

max_modis = ds[model_var+'_modis'].quantile(0.95, dim='time').drop('quantile')
max_modis = max_modis.expand_dims(time=ds.time)
ds[model_var+'_modis_max'] = max_modis

#remove unneeded variables
ds = ds.drop([model_var+'_stddev', 'n_obs', model_var+'_cv'])

### Add derivative of rainfall

In [ ]:
# derived = ds['rain'].differentiate(coord='time', datetime_unit='D')
# ds['rain_gradient'] = derived
ds

### Mask for trees/no-trees mask

In [ ]:
# if model_var=='NDVI'
trees = xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/data/5km/WCF_5km_monthly_1982_2022.nc')['WCF']
trees = assign_crs(trees, crs ='epsg:4326')
trees = trees.sel(time=slice('2001', '2018'))
trees = trees.mean('time')

if name=='trees':
    mask = xr.where(trees>0.25, 1, 0)
if name=='nontrees':
    mask = xr.where(trees<=0.25, 1, 0)

In [ ]:
ds = ds.where(filtered_mask) # mask covariables with avhhr filtered
ds = ds.where(mask) # mask trees/notrees

## Training & testing data: equal random sampling of bioclimatic regions

Can skip to importing TD if already run this

In [ ]:
gdf = gpd.read_file('/g/data/os22/chad_tmp/NEE_modelling/data/bioclimatic_regions.geojson')

In [ ]:
# Dictionary to save results 
results = []
for index, row in gdf.iterrows():

    if name=='trees':
        n=int(n_samples/5)
    
    if name=='nontrees':
        n=int(n_samples/6)
        
    if (name=='trees') & (row['region_name']=='Desert'):
        continue

    # else:
    print(row['region_name'], n)
    
    # Generate a polygon mask to keep only data within the polygon
    mask = xr_rasterize(gdf.iloc[[index]], ds)
    mask = round_coords(mask)
    
    # Mask dataset to set pixels outside the polygon to `NaN`
    dss = ds.sel(time=slice('2000-03','2013')).where(mask)
    
    #sample equivalent num of samples per region
    df = dss.to_dataframe().dropna().sample(n=n, random_state=0).reset_index()
    
    # Append results to a dictionary using the attribute
    # column as an key
    results.append(df)


In [ ]:
df = pd.concat(results).reset_index(drop=True)
df['year'] = pd.DatetimeIndex(df['time']).year

In [ ]:
df = df.drop(['time','spatial_ref'], axis=1)

### Independent validation samples

In [ ]:
validation = df.sample(n=n_val, random_state=0)

In [ ]:
df = df.drop(validation.index)
print(len(df), 'training samples')

### Plot the location of the samples

In [ ]:
gdf = gpd.GeoDataFrame(
    df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326"
)

gdf_val = gpd.GeoDataFrame(
    validation, geometry=gpd.points_from_xy(validation.longitude, validation.latitude), crs="EPSG:4326"
)

gdf.to_file('/g/data/os22/chad_tmp/climate-carbon-interactions/data/'+model_var+'_harmonization/'+model_var+'_'+name+'_training_data.geojson')
gdf_val.to_file('/g/data/os22/chad_tmp/climate-carbon-interactions/data/'+model_var+'_harmonization/'+model_var+'_'+name+'_validation_data.geojson')

In [ ]:
gdf = gdf.drop(['longitude'], axis=1)
gdf_val = gdf_val.drop(['longitude'], axis=1)

# gdf.explore(column='year', cmap='inferno')
gdf_val.plot(column='year', figsize=(5,5), markersize=1)

In [ ]:
# gdf = gpd.read_file('/g/data/os22/chad_tmp/climate-carbon-interactions/data/NDVI_harmonization/'+model_var+'_training_data.geojson')
# gdf_val = gpd.read_file('/g/data/os22/chad_tmp/climate-carbon-interactions/data/NDVI_harmonization/'+model_var+'_validation_data.geojson')

### Prepare data for training

In [ ]:
df = pd.DataFrame(gdf.drop(columns='geometry', axis=1))
validation = pd.DataFrame(gdf_val.drop(columns='geometry', axis=1))

df = df.drop(['year'], axis=1)
validation = validation.drop(['year'], axis=1)

y = df[model_var+'_modis']
x = df.drop([model_var+'_modis'], axis=1)

## Testing model using nested CV

In [ ]:
# Create the parameter grid using distributions
param_grid = {
    'num_leaves': stats.randint(5,50),
    'min_child_samples':stats.randint(10,30),
    'boosting_type': ['gbdt', 'dart'],
    'max_depth': stats.randint(5,25),
    'n_estimators': [300, 400, 500],
}

In [ ]:
outer_cv = KFold(n_splits=5, shuffle=True,
                   random_state=0)

# lists to store results of CV testing
acc = []
rmse=[]
r2=[]

i = 1
for train_index, test_index in outer_cv.split(x, y):
    print(f"Working on {i}/5 outer CV split", end='\r')
    model = LGBMRegressor(random_state=1,
                          verbose=-1,
                          # n_jobs=-1
                          )

    # index training, testing
    X_tr, X_tt = x.iloc[train_index, :], x.iloc[test_index, :]
    y_tr, y_tt = y.iloc[train_index], y.iloc[test_index]
    
    #simple random split on inner fold
    inner_cv = KFold(n_splits=3,
                     shuffle=True,
                     random_state=0)
    
    clf = RandomizedSearchCV(
                   model,
                   param_grid,
                   verbose=0,
                   n_iter=50,
                   # n_jobs=-1,
                   cv=inner_cv.split(X_tr, y_tr)
                  )
    
    #prevents extensive print statements
    clf.fit(X_tr, y_tr, callbacks=None)
    
    # predict using the best model
    best_model = clf.best_estimator_
    pred = best_model.predict(X_tt)

    # evaluate model w/ multiple metrics
    # r2
    r2_ = r2_score(y_tt, pred)
    r2.append(r2_)
    # MAE
    ac = mean_absolute_error(y_tt, pred)
    acc.append(ac)
    # RMSE
    rmse_ = np.sqrt(mean_squared_error(y_tt, pred))
    rmse.append(rmse_)
    
    #1:1 plots for each fold (save to csv so we can make a plot later on)
    df = pd.DataFrame({'Test':y_tt, 'Pred':pred}).reset_index(drop=True)

    df.to_csv("/g/data/os22/chad_tmp/climate-carbon-interactions/data/"+model_var+"_harmonization/cross_validation/"+str(i)+"_"+model_var+"_lgbm_"+name+".csv")
    
    i += 1

### Create a single 1:1 plot out of the folds 


In [ ]:
dffs=[]
for i in range(1,5+1):
    df = pd.read_csv("/g/data/os22/chad_tmp/climate-carbon-interactions/data/"+model_var+"_harmonization/cross_validation/"+str(i)+"_"+model_var+"_lgbm_"+name+".csv",
                     usecols=['Test', 'Pred'])
    dffs.append(df)

cross_df = pd.concat(dffs)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(5,5))

xy = np.vstack([cross_df['Test'],cross_df['Pred']])
z = gaussian_kde(xy)(xy)

sb.scatterplot(data=cross_df, x='Test',y='Pred', cmap='magma', c=z, s=10, lw=1, alpha=0.5, ax=ax)
sb.regplot(data=cross_df, x='Test',y='Pred', scatter=False, color='darkblue', ax=ax)
sb.regplot(data=cross_df, x='Test',y='Test', color='black', scatter=False, line_kws={'linestyle':'dashed'}, ax=ax)
# plt.colorbar()
plt.xlabel('MODIS '+model_var, fontsize=16)
plt.ylabel('Prediction '+model_var, fontsize=16)
ax.text(.05, .95, 'r\N{SUPERSCRIPT TWO}={:.2f}'.format(np.mean(r2)),
            transform=ax.transAxes, fontsize=16)
ax.text(.05, .9, 'MAE={:.2g}'.format(np.mean(acc)),
            transform=ax.transAxes, fontsize=16)

if model_var=='NDVI':
    ax.set_ylim(0.0, 0.95)
    ax.set_xlim(0.0, 0.95)

ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

plt.tight_layout()
fig.savefig("/g/data/os22/chad_tmp/climate-carbon-interactions/data/"+model_var+"_harmonization/cross_validation/cross_val_"+model_var+"_lgbm_"+name+".png",
            bbox_inches='tight', dpi=300)

### Optimize model using all training data

Using a randomized strategy so we can search through more variables, with 500 iterations


In [ ]:
outer_cv = KFold(n_splits=5, shuffle=True,
                   random_state=0)

clf = RandomizedSearchCV(LGBMRegressor(verbose=-1),
                   param_grid,
                   verbose=1,
                   n_iter=250,
                   # n_jobs=-1,
                   cv=outer_cv
                  )

clf.fit(x, y, callbacks=None)

In [ ]:
print("The most accurate combination of tested parameters is: ")
pprint(clf.best_params_)
print('\n')
print("The best score using these parameters is: ")
print(round(clf.best_score_, 2))

### Fit on all data using best params

In [ ]:
model = LGBMRegressor(**clf.best_params_)

model.fit(x,y)

## Compare with independent validation data

In [ ]:
y_val = validation[model_var+'_modis']
x_val = validation.drop([model_var+'_modis'], axis=1)

In [ ]:
pred = model.predict(x_val)

r2 = r2_score(y_val, pred)
ac = mean_absolute_error(y_val, pred)
df_val = pd.DataFrame({'Test':y_val, 'Pred':pred}).reset_index(drop=True)

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(5,5))

xy = np.vstack([df_val['Test'],df_val['Pred']])
z = gaussian_kde(xy)(xy)

sb.scatterplot(data=df_val, x='Test',y='Pred', cmap='magma', c=z, s=20, lw=1, alpha=0.5, ax=ax, legend=True)
sb.regplot(data=df_val, x='Test',y='Pred', scatter=False, color='darkblue', ax=ax)
sb.regplot(data=df_val, x='Test',y='Test', color='black', scatter=False, line_kws={'linestyle':'dashed'}, ax=ax);

plt.xlabel('MODIS '+model_var, fontsize=16)
plt.ylabel('Prediction '+model_var, fontsize=16)
ax.text(.05, .95, 'R\N{SUPERSCRIPT TWO}={:.2f}'.format(r2),
            transform=ax.transAxes, fontsize=16)
ax.text(.05, .9, 'MAE={:.2g}'.format(ac),
            transform=ax.transAxes, fontsize=16)

if model_var=='NDVI':
    ax.set_ylim(0.05, 0.95)
    ax.set_xlim(0.05, 0.95)

ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

plt.tight_layout()
fig.savefig("/g/data/os22/chad_tmp/climate-carbon-interactions/data/"+model_var+"_harmonization/cross_validation/validation_"+model_var+"_lgbm_"+name+".png",
            bbox_inches='tight', dpi=300)

### Save the model

In [ ]:
dump(model, '/g/data/os22/chad_tmp/climate-carbon-interactions/data/'+model_var+'_harmonization/Harmonization_LGBM_'+name+'.joblib')

### Examine feature importance using SHAP


In [ ]:
# explain the model's predictions using SHAP
explainer = shap.Explainer(model)
shap_values = explainer(x)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(5,7))
shap.plots.violin(shap_values, max_display=10, show=False, plot_type="layered_violin")
ax = plt.gca() 
plt.gcf().axes[-1].set_aspect('auto')
plt.gcf().axes[-1].set_box_aspect(15)
ax.tick_params(axis='x', labelsize=16)
ax.tick_params(axis='y', labelsize=16)

if name=='trees':
    ax.set_xlabel(model_var+ ' "Trees" '+'SHAP Value', fontsize=16)
    ax.set_xlim(-0.1, 0.1)
if name=='nontrees':
    ax.set_xlabel(model_var+ ' "Non-Trees" '+'SHAP Value', fontsize=16)
    ax.set_xlim(-0.2, 0.2)
plt.tight_layout()
fig.savefig("/g/data/os22/chad_tmp/climate-carbon-interactions/data/"+model_var+"_harmonization/cross_validation/feature_importance_"+model_var+"_lgbm_"+name+".png",
            bbox_inches='tight', dpi=300)

## Predict

In [ ]:
from datacube.utils.dask import start_local_dask

import sys
sys.path.append('/g/data/os22/chad_tmp/dea-notebooks/Tools/')
from dea_tools.classification import predict_xr, HiddenPrints

In [ ]:
client = start_local_dask(mem_safety_margin='2Gb')
client

### Load model

In [ ]:
# model = load('/g/data/os22/chad_tmp/climate-carbon-interactions/data/NDVI_harmonization/Harmonization_LGBM_'+name+'.joblib').set_params(n_jobs=1)

### Add lat as a variable

Plus ensure order of the variables is correct for predictions

In [ ]:
y = ds.latitude
y = y.expand_dims(time=ds.time, longitude=ds.longitude)
y = y.transpose('time', 'latitude', 'longitude')
ds['latitude_data'] = y

# lon = ds.longitude
# lon = lon.expand_dims(time=ds.time, latitude=ds.latitude)
# lon = lon.transpose('time', 'latitude', 'longitude')
# ds['longitude_gridded'] = lon

### Ensure features match

In [ ]:
columns = list(ds.data_vars)[:-1]
columns.insert(0, 'latitude_data')
ds = ds[columns]
ds = ds.drop(model_var+'_modis')

In [ ]:
train_vars = list(x.columns)
train_vars[0] = 'latitude_data'

if train_vars == list(ds.data_vars):
    print('Variables match, n: ', len(ds.data_vars))
else:
    raise ValueError('Variables dont match')

### Re-create the mask

In [ ]:
trees = xr.open_dataset('/g/data/os22/chad_tmp/climate-carbon-interactions/data/5km/WCF_5km_monthly_1982_2022.nc')['WCF']
trees = assign_crs(trees, crs ='epsg:4326')
trees=trees.sel(time=slice('2001', '2018'))
trees = trees.mean('time')

if name=='trees':
    mask = xr.where(trees>0.25, 1, 0)
if name=='nontrees':
    mask = xr.where(trees<=0.25, 1, 0)

#rename so predict_xr is happy
ds = ds.rename({'latitude':'y', 'longitude':'x'})

### Predict

In [ ]:
warnings.filterwarnings("ignore")
results = []
i=0
for i in range(0, len(ds.time)):
    print(" {:03}/{:03}\r".format(i + 1, len(range(0, len(ds.time)))), end="")
    with HiddenPrints():
        predicted = predict_xr(model,
                            ds.isel(time=i).drop('time'),
                            proba=False,
                            clean=True,
                            chunk_size=100000,
                              ).compute()
    
    # predicted = predicted.Predictions.where(~mask.isel(time=i))
    predicted = predicted.assign_coords(time=ds.isel(time=i).time).expand_dims(time=1)
    results.append(predicted.astype('float32'))
    i+=1

In [ ]:
yy = xr.concat(results, dim='time').sortby('time').rename({'Predictions':model_var})#.astype('float32')
yy = yy.where(mask.rename({'latitude':'y', 'longitude':'x'}))
yy = yy.where(filtered_mask.rename({'latitude':'y', 'longitude':'x'}))

In [ ]:
yy.to_netcdf('/g/data/os22/chad_tmp/climate-carbon-interactions/data/'+model_var+'_harmonization/'+model_var+'_'+name+'_LGBM_harmonize_5km_monthly_1982_2013.nc')

In [ ]:
yy[model_var].mean(['x', 'y']).plot(figsize=(11,4))
# plt.ylim(0.2,0.38)